In [2]:
import pandas as pd
import math
import numpy as np
from geopy.distance import geodesic

# First solution

In [3]:
# Function to find the closest unvisited city using geodesic distance
def find_closest_city(df, start_coord, visited_cities):
    # Initialize variables to store the closest city
    min_distance = float('inf')
    closest_city = None
    closest_city_coord = None
    
    # Loop through each city to calculate the geodesic distance
    for index, row in df.iterrows():
        city_name = row[0]
        city_coord = (row[1], row[2])  # Assuming columns [1] and [2] are latitude and longitude
        
        # Skip the city if it's already visited
        if city_name in visited_cities:
            continue
        
        # Calculate geodesic distance between start_coord and city_coord
        distance = geodesic(start_coord, city_coord).kilometers
        
        # Update the closest city if this one is closer
        if distance < min_distance and distance != 0:
            min_distance = distance
            closest_city = city_name
            closest_city_coord = city_coord
    
    return closest_city, closest_city_coord, min_distance
    

In [11]:
# Function to solve the TSP using geodesic distance and nearest neighbor
def tsp_nearest_neighbor(filename):
    # Read CSV
    df = pd.read_csv(filename)
    
    # Initialize visited cities array
    visited_cities = []
    
    # Select a random starting city
    start_index = np.random.choice(df.index)
    start_city = df.iloc[start_index][0]
    start_city_coord = (df.iloc[start_index][1], df.iloc[start_index][2])
    
    # Add starting city to the visited list
    visited_cities.append(start_city)
    
    print(f"Starting city: {start_city}")
    
    total_distance = 0
    current_city_coord = start_city_coord
    
    # Loop until all cities are visited
    while len(visited_cities) < len(df):
        # Find the closest unvisited city
        closest_city, closest_city_coord, min_distance = find_closest_city(df, current_city_coord, visited_cities)
        
        # Add the closest city to the visited cities array
        visited_cities.append(closest_city)
        
        # Update the current city coordinates to the new closest city's coordinates
        current_city_coord = closest_city_coord
        
        # Accumulate the total distance
        total_distance += min_distance
        
        print(f"Travels to {closest_city} with a distance of {min_distance:.2f} km")
    
    # Return to the start city to complete the loop
    return_to_start_distance = geodesic(current_city_coord, start_city_coord).kilometers
    total_distance += return_to_start_distance
    visited_cities.append(start_city)  # Add the starting city at the end to close the loop
    
    print(f"Returning to {start_city} with a distance of {return_to_start_distance:.2f} km")

    
    return visited_cities, total_distance

In [17]:
# Vanuatu
visited_cities, total_distance = tsp_nearest_neighbor('vanuatu.csv')
print('---------------------------------------------------')
print(f"Total distance traveled: {total_distance:.2f} km")
print(f"Visited cities in order: {visited_cities}")

Starting city: Port Olry
Travels to Luganville with a distance of 52.02 km
Travels to Norsup with a distance of 67.09 km
Travels to Lakatoro with a distance of 2.46 km
Travels to Longana with a distance of 108.53 km
Travels to Sola with a distance of 165.49 km
Travels to Vila with a distance of 435.91 km
Returning to Port Olry with a distance of 326.69 km
---------------------------------------------------
Total distance traveled: 1158.19 km
Visited cities in order: ['Port Olry', 'Luganville', 'Norsup', 'Lakatoro', 'Longana', 'Sola', 'Vila', 'Port Olry']


# Second solution